In [16]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
# things we need for Tensorflow
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import pandas as pd
import pickle
import random
import json
import tensorflow as tf

In [5]:
with open("intents.json") as file:
	intents = json.load(file)

In [6]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique stemmed words", words)


188 documents
44 classes ['Abdonominal Pain', 'Abrasions', 'Broken Toe', 'Bruises', 'CPR', 'Chemical Burn', 'Choking', 'Cold', 'Cough', 'Cuts', 'Diarrhea', 'Drowning', 'Eye Injury', 'Fainting', 'Fever', 'Fracture', 'Frost bite', 'Gastrointestinal problems', 'Head Injury', 'Headache', 'Heat Exhaustion', 'Heat Stroke', 'Insect Bites', 'Nasal Congestion', 'Normal Bleeding', 'Poison', 'Pulled Muscle', 'Rash', 'Rectal bleeding', 'Skin problems', 'Sore Throat', 'Splinter', 'Sprains', 'Strains', 'Sun Burn', 'Teeth', 'Testicle Pain', 'Vertigo', 'Wound', 'animal bite', 'nose bleed', 'seizure', 'snake bite', 'stings']
114 unique stemmed words ['a', 'abdonomin', 'abras', 'allergy', 'am', 'an', 'anim', 'apply', 'bet', 'bit', 'blee', 'block', 'bring', 'brok', 'bru', 'burn', 'by', 'caus', 'chem', 'chok', 'cold', 'congest', 'cough', 'cpr', 'cream', 'cur', 'cut', 'diagnos', 'diarrhe', 'do', 'doe', 'dog', 'drown', 'due', 'exhaust', 'ey', 'faint', 'feel', 'fev', 'for', 'fract', 'frost', 'gas', 'gastroin

In [7]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word - create base word, in attempt to represent related words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])

C:\Users\Leinad\AppData\Local\Temp\ipykernel_5948\1377439408.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [8]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [10]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [11]:
# Fit the model
model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
38/38 [==============================] - 2s 5ms/step - loss: 3.8205 - accuracy: 0.0053
Epoch 2/200
38/38 [==============================] - 0s 4ms/step - loss: 3.7891 - accuracy: 0.0213
Epoch 3/200
38/38 [==============================] - 0s 6ms/step - loss: 3.7513 - accuracy: 0.0372
Epoch 4/200
38/38 [==============================] - 0s 4ms/step - loss: 3.6861 - accuracy: 0.0691
Epoch 5/200
38/38 [==============================] - 0s 4ms/step - loss: 3.6602 - accuracy: 0.0745
Epoch 6/200
38/38 [==============================] - 0s 4ms/step - loss: 3.6179 - accuracy: 0.0851
Epoch 7/200
38/38 [==============================] - 0s 3ms/step - loss: 3.5585 - accuracy: 0.0745
Epoch 8/200
38/38 [==============================] - 0s 4ms/step - loss: 3.3631 - accuracy: 0.1755
Epoch 9/200
38/38 [==============================] - 0s 4ms/step - loss: 3.2882 - accuracy: 0.1915
Epoch 10/200
38/38 [==============================] - 0s 4ms/step - loss: 3.0838 - accuracy: 0.2181
Epoch 11/

In [13]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [14]:
p = bow("Load blood pessure for patient", words)
print (p)
print (classes)

found in bag: for
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]
['Abdonominal Pain', 'Abrasions', 'Broken Toe', 'Bruises', 'CPR', 'Chemical Burn', 'Choking', 'Cold', 'Cough', 'Cuts', 'Diarrhea', 'Drowning', 'Eye Injury', 'Fainting', 'Fever', 'Fracture', 'Frost bite', 'Gastrointestinal problems', 'Head Injury', 'Headache', 'Heat Exhaustion', 'Heat Stroke', 'Insect Bites', 'Nasal Congestion', 'Normal Bleeding', 'Poison', 'Pulled Muscle', 'Rash', 'Rectal bleeding', 'Skin problems', 'Sore Throat', 'Splinter', 'Sprains', 'Strains', 'Sun Burn', 'Teeth', 'Testicle Pain', 'Vertigo', 'Wound', 'animal bite', 'nose bleed', 'seizure', 'snake bite', 'stings']


In [17]:
# Use pickle to load in the pre-trained model
global graph
graph = tf.get_default_graph()
with open(f'katana-assistant-model.pkl', 'rb') as f:
    model = pickle.load(f)

AttributeError: module 'tensorflow' has no attribute 'get_default_graph'